Importación y Carga de Datos

In [17]:
# Importaciones
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('telecom_customer_churn.csv')

Preprocesamiento

In [18]:
# 1. Preprocesamiento
# a. Eliminar entradas con el estado "Joined"
df = df[df['Customer Status'] != 'Joined']

# b. Crear la columna 'Churn' y asignar 1 si 'Customer Status' es 'Churned', de lo contrario 0
df['Churn'] = df['Customer Status'].apply(lambda x: 1 if x == 'Churned' else 0)

# c. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# d. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# e. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Customer Status':
        # Llenar valores faltantes con la moda para columnas categóricas
        df[column].fillna(df[column].mode()[0], inplace=True)
        # Convertir variables categóricas a numéricas
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        # Llenar valores faltantes con la mediana para columnas numéricas
        df[column].fillna(df[column].median(), inplace=True)

# Eliminar la columna 'Customer Status' antes de normalizar
df = df.drop(columns=['Customer Status'])

# f. Normalización
# Seleccionar solo las columnas que no son 'Churn' para la normalización
cols_to_scale = df.columns.difference(['Churn'])
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

Balanceo con SMOTE

In [19]:
# 3. Balanceo con SMOTE
X = df.drop('Churn', axis=1)
y = df['Churn']
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

División de ConjuntoBalanceo

In [20]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

Entrenamiento de ANN

In [21]:
# 5. Entrenamiento de ANN con búsqueda de hiperparámetros usando Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=64, step=8), 
                    input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=64, step=8), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=100,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='keras_tuner_example'
)

# Ajusta el callback de ModelCheckpoint para guardar en el nuevo formato
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
]

# Ejecutar la búsqueda de hiperparámetros
tuner.search(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks)

# Obtener los mejores hiperparámetros.
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(X_train.shape)  # Esto te mostrará las dimensiones actuales de entrada

# Imprimir los mejores hiperparámetros
print(f"""
Los mejores hiperparámetros encontrados:
Número de unidades: {best_hps.get('units')}
Tasa de aprendizaje: {best_hps.get('learning_rate')}
""")

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

Trial 24 Complete [00h 00m 05s]
val_accuracy: 0.8398411870002747

Best val_accuracy So Far: 0.8524156212806702
Total elapsed time: 00h 02m 04s
(7552, 35)

Los mejores hiperparámetros encontrados:
Número de unidades: 64
Tasa de aprendizaje: 0.01



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Evaluación del modelo

In [22]:
# 6. Evaluación del modelo con los mejores hiperparámetros
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Obtener las predicciones en forma de clases
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")

# Calcular la matriz de confusión, y generar el reporte de clasificación
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Imprimir las métricas
print(conf_matrix)
print("Reporte de clasificación:\n", report)

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8465 - loss: 0.3470
Loss: 0.33353692293167114, Accuracy: 0.8516949415206909
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step
[[790 164]
 [116 818]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.87      0.83      0.85       954
           1       0.83      0.88      0.85       934

    accuracy                           0.85      1888
   macro avg       0.85      0.85      0.85      1888
weighted avg       0.85      0.85      0.85      1888



Validación cruzada

In [23]:
from sklearn.model_selection import KFold
import numpy as np

# 7. Validación Cruzada
# Asegúrate de tener los mejores hiperparámetros disponibles, 'best_hps', que deberían haber sido obtenidos de tu optimización con Keras Tuner
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

def create_best_model():
    model = Sequential()
    model.add(Dense(units=best_hps.get('units'), input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(units=best_hps.get('units'), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=best_hps.get('learning_rate'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Configuración de KFold para la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para guardar las puntuaciones de cada fold
accuracy_scores = []

for train_index, test_index in kf.split(X_train):
    # Dividir datos en entrenamiento y prueba para este fold usando .iloc para DataFrames de pandas
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Crear y compilar el modelo
    model = create_best_model()

    # Entrenar el modelo
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=10, verbose=0)

    # Evaluar el modelo en el conjunto de prueba del fold
    _, accuracy = model.evaluate(X_test_fold, y_test_fold, verbose=0)
    accuracy_scores.append(accuracy)

# Calcular la media de las puntuaciones de precisión
average_accuracy = np.mean(accuracy_scores)
print(f"Accuracy promedio con Validación Cruzada Manual: {average_accuracy}")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy promedio con Validación Cruzada Manual: 0.8673194885253906
